In [1]:
import random

In [1]:
import pennylane as qml
from pennylane import numpy as np
import itertools, math, random

def bao_qaoa_resources(angles, k_beams=5, p=2, shots=1000,
                       weights=None, seed=17, train_steps=200):
    """
    Build and (optionally) train a p-layer QAOA circuit for Beam-Angle
    Optimisation and return a resource-estimation dictionary.
    ----------
    angles : iterable of gantry degrees
    k_beams: exact number of beams to select
    p      : QAOA depth (layers)
    shots  : device shots for sampling (set None for analytic)
    weights: dict of clinical weights
             {W_NTCP, W_TCP, FLASH_BEN, FLASH_HARM, SETUP, TIME}
    seed   : RNG seed
    train_steps : Adam iterations (0 ⇒ skip training)
    """
    # ---------- default weights ------------------------------------
    if weights is None:
        weights = dict(W_NTCP=1.0, W_TCP=-0.5,
                       FLASH_BEN=-2.0, FLASH_HARM=3.0,
                       SETUP=200, TIME=-5.0)
    W_NTCP, W_TCP   = weights["W_NTCP"], weights["W_TCP"]
    F_B, F_H        = weights["FLASH_BEN"], weights["FLASH_HARM"]
    SETUP, TIME_VAL = weights["SETUP"], weights["TIME"]
    time_saved_flash = 4

    # ---------- per-beam surrogate cost c_i ------------------------
    def depth(d):                              # crude bucket model
        if d in range(330,360) or d<30: return 6
        if 60<d<120 or 240<d<300:          return 10
        return 14
    def r_max(dep): return 90 if dep<=5 else (90-6*(dep-5) if dep<=15 else 30)
    def c_i(deg):
        d,r = depth(deg), r_max(depth(deg))
        return (W_NTCP*(d>8) + W_TCP*(d<12) +
                (F_B if r>=40 else F_H)      +
                (SETUP if r>=40 else 0)      +
                (TIME_VAL*time_saved_flash if r>=40 else 0))
    c = np.array([c_i(a) for a in angles])

    # ---------- QUBO -----------------------------------------------
    M = 10*abs(c).max()
    n = len(angles)
    Q = np.zeros((n,n))
    for i in range(n):
        Q[i,i] = c[i] + M*(1-2*k_beams)
        for j in range(i+1,n):
            Q[i,j] = Q[j,i] = M

    # ---------- Ising coefficients ---------------------------------
    coeffs, ops = [], []
    for i in range(n):
        coeffs.append(0.5*Q[i,i]); ops.append(qml.PauliZ(i))
    for i in range(n):
        for j in range(i+1,n):
            coeffs.append(0.25*Q[i,j]); ops.append(qml.PauliZ(i)@qml.PauliZ(j))
    H = qml.Hamiltonian(coeffs, ops) / M   # scale for stable grads

    # ---------- QAOA circuit & (optional) training -----------------
    dev = qml.device("default.qubit", wires=n, shots=shots)
    @qml.qnode(dev)
    def circuit(params):
        for w in range(n): qml.Hadamard(wires=w)
        gam, bet = params[:p], params[p:]
        for l in range(p):
            qml.ApproxTimeEvolution(H, gam[l], 1)
            for w in range(n): qml.RX(2*bet[l], wires=w)
        return qml.expval(H)

    np.random.seed(seed)
    params = np.random.rand(2*p)*np.pi
    if train_steps:
        opt = qml.AdamOptimizer(0.04)
        for _ in range(train_steps):
            params = opt.step(circuit, params)

    # ---------- simple resource report -----------------------------
    depth_per_layer = 1 + 1          #  AppTimeEvolution + RX mix
    total_depth = p*depth_per_layer
    two_qubit_ops = len([op for op in ops if len(op.wires)==2])
    report = dict(
        qubits          = n,
        qaoa_depth_p    = p,
        circuit_depth   = total_depth,
        two_qubit_ops   = two_qubit_ops,
        total_parameters= 2*p,
        expectation_val = float(circuit(params)*M),  # un-scaled cost
        penalty_M       = M,
    )
    return report, params, Q, H
